<a href="https://colab.research.google.com/github/Zypher44/RaduIntuneric/blob/master/RNN_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7dcaca7e-9f0f-f863-55bf-f3218ef55482)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-22 00:27:21--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-22 00:27:21 (102 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import *

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2021-07-22 00:27:58--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 142.251.2.128, 74.125.137.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-07-22 00:27:58 (147 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
train_df.head()

NameError: ignored

In [6]:
test_df.head()

NameError: ignored

In [7]:
import pandas as pd 
train_df = pd.read_csv("train.csv")
test_df= pd.read_csv("test.csv")
train_df.shape

(7613, 5)

In [8]:
test_df.shape

(3263, 4)

In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)

In [11]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [12]:
from sklearn.model_selection import train_test_split
train_senteces, val_senteces, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42) 


In [13]:
len(train_senteces), len(val_senteces)

(6851, 762)

In [14]:
len(train_labels), len(val_labels)

(6851, 762)

In [15]:
train_senteces[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

#Tokenizing data 

In [16]:
import tensorflow as tf 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)

In [17]:
#Find the average number of tokens in training 

max_length = round(sum([len(i.split()) for i in train_senteces])/len(train_senteces))
max_length

15

In [18]:
max_vocab_length = 10000
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [19]:
text_vectorizer.adapt(train_senteces)

In [20]:
random_sentence = ["The cat with the gat"]

text_vectorizer(["The cat with the gat"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 1427,   14,    2,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [21]:
#find unique words 

unique_words = text_vectorizer.get_vocabulary()
top5_words = unique_words[:5]
top5_words

['', '[UNK]', 'the', 'a', 'in']

In [22]:
bottom5 = unique_words[-5:]
bottom5

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [23]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                      output_dim=128,
                                      input_length=max_length)

In [24]:
embedding

In [25]:
sample_embed = embedding(text_vectorizer(['THe cat with the gat']))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01958549, -0.00665141,  0.0100596 , ...,  0.01393143,
          0.04290083,  0.01379908],
        [ 0.04111535,  0.03904602, -0.04096695, ..., -0.04007198,
          0.03167165, -0.03649616],
        [-0.02009262, -0.04950454,  0.00094458, ..., -0.02281857,
          0.03570792, -0.01166046],
        ...,
        [ 0.01280713,  0.00746732, -0.01230075, ..., -0.03203064,
         -0.01534704,  0.03240642],
        [ 0.01280713,  0.00746732, -0.01230075, ..., -0.03203064,
         -0.01534704,  0.03240642],
        [ 0.01280713,  0.00746732, -0.01230075, ..., -0.03203064,
         -0.01534704,  0.03240642]]], dtype=float32)>

In [26]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.Input(shape=(1,),dtype=tf.string))
model1.add(text_vectorizer)
model1.add(embedding)
model1.add(tf.keras.layers.GlobalMaxPool1D())
model1.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [27]:
model1.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [28]:
history1 = model1.fit(
    x=train_senteces,
    y=train_labels,
    epochs=5,
    validation_data=(val_senteces,val_labels),
    callbacks=[create_tensorboard_callback("Save.dir",
                                           "Squentialmodel")]
)

Saving TensorBoard log files to: Save.dir/Squentialmodel/20210722-002941
Epoch 1/5
215/215 [==============================] - 7s 17ms/step - loss: 0.6363 - accuracy: 0.6494 - val_loss: 0.5721 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4681 - accuracy: 0.8321 - val_loss: 0.4720 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3406 - accuracy: 0.8742 - val_loss: 0.4502 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2579 - accuracy: 0.9069 - val_loss: 0.4519 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1980 - accuracy: 0.9327 - val_loss: 0.4674 - val_accuracy: 0.7887


In [29]:
model1.evaluate(val_senteces,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4674 - accuracy: 0.7887


[0.46742525696754456, 0.7887139320373535]

In [30]:
model1_pred_probs = model1.predict(val_senteces)
model1_pred_probs.shape,model1_pred_probs[:10]


((762, 1), array([[0.36080417],
        [0.8391482 ],
        [0.9963386 ],
        [0.05265369],
        [0.31308448],
        [0.97059107],
        [0.93624604],
        [0.98745894],
        [0.9431592 ],
        [0.08415614]], dtype=float32))

In [31]:
model1_preds= tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [32]:
model1_results = calculate_results(y_true=val_labels,
                                   y_pred= model1_preds)
model1_results

{'accuracy': 78.87139107611549,
 'f1': 0.7863615341411515,
 'precision': 0.7919183386292588,
 'recall': 0.7887139107611548}

In [33]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


model2

In [34]:
from tensorflow.keras import layers

model2 = tf.keras.Sequential()
model2.add(layers.Input(shape=(1,),dtype=tf.string))
model2.add(text_vectorizer)
model2.add(embedding)
model2.add(layers.LSTM(units=64,return_sequences=True))
model2.add(layers.LSTM(units=64,activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))



In [35]:
model2.compile(
    loss= tf.keras.losses.BinaryCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(),
    metrics=['accuracy']
  )

In [36]:
history2 = model2.fit(
    x=train_senteces,
    y=train_labels,
    epochs=5,
    validation_data=(val_senteces,val_labels),
    callbacks=[create_tensorboard_callback("Save.dir",
                                           "Model2Seq")]
)

Saving TensorBoard log files to: Save.dir/Model2Seq/20210722-003248
Epoch 1/5
215/215 [==============================] - 16s 49ms/step - loss: 0.3828 - accuracy: 0.8351 - val_loss: 0.4887 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.2447 - accuracy: 0.9018 - val_loss: 0.5865 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 9s 44ms/step - loss: 0.1760 - accuracy: 0.9320 - val_loss: 0.6661 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 9s 42ms/step - loss: 0.1260 - accuracy: 0.9510 - val_loss: 0.7992 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 9s 41ms/step - loss: 0.1028 - accuracy: 0.9593 - val_loss: 0.7882 - val_accuracy: 0.7730


In [37]:
model2.evaluate(val_senteces,val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.7882 - accuracy: 0.7730


[0.7881549000740051, 0.7729659080505371]

In [38]:
model2_pred_probs = model2.predict(val_senteces)
model2_pred_probs[:10]

array([[0.4849052 ],
       [0.77379686],
       [0.9999553 ],
       [0.21419463],
       [0.00168768],
       [0.9999999 ],
       [0.80515903],
       [0.99999976],
       [0.99999857],
       [0.35561007]], dtype=float32)

In [39]:
model2_preds = tf.squeeze(tf.round(model2_pred_probs))
model2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [40]:
model2_results = calculate_results(y_true=val_labels,
                                   y_pred=model2_preds
                                   )
model2_results

{'accuracy': 77.29658792650919,
 'f1': 0.7717707641179984,
 'precision': 0.7731790030340066,
 'recall': 0.7729658792650919}

#Get 10% of the data

In [57]:
train_10_percent_split = int(0.1*len(train_senteces))
train_senteces_10_percent = train_senteces[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


baseline = Pipeline([
                     ('tfidf',TfidfVectorizer()),
                     ("clf", MultinomialNB())
])

In [61]:
#Fit 
baseline.fit(train_senteces_10_percent,
             train_labels_10_percent)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [62]:
baselinescore = baseline.score(val_senteces,val_labels)

In [63]:
baselinescore

0.7020997375328084

In [66]:
import tensorflow_hub as hub 
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed(["theCatwith the Gat"])
embed_sample

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 8.82856697e-02, -2.74656974e-02,  1.79847479e-02,
         5.05848117e-02,  8.64770561e-02, -3.13652642e-02,
         3.39986347e-02, -1.25354476e-04,  6.41684700e-03,
         5.46488538e-02,  1.03832381e-02, -6.05557598e-02,
        -5.65233976e-02, -6.76394254e-02,  1.91923860e-03,
        -3.34322988e-03,  2.02487316e-02, -1.29964892e-02,
         8.18769485e-02, -8.34186524e-02, -2.40566656e-02,
         8.92516412e-03,  4.47505936e-02,  1.21476166e-02,
         5.30343922e-03, -1.99864507e-02,  2.09703352e-02,
        -3.93814109e-02,  1.03836348e-02,  4.12788521e-03,
         2.04428304e-02, -3.70127261e-02,  4.02830876e-02,
        -6.14776500e-02,  1.63373128e-02, -3.05093396e-02,
        -5.82643896e-02, -1.43260844e-02, -7.63395578e-02,
         2.70195697e-02, -7.14580342e-03, -1.04716858e-02,
         1.80133022e-02, -3.15065086e-02,  2.17051040e-02,
         6.57813298e-03,  1.95489861e-02, -4.38746661e-02,
      

In [98]:
sentence_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                  input_shape=[],
                                  dtype="string",
                                  name="USE",
                                  trainable=True)

In [90]:
model3 = tf.keras.Sequential([
    sentence_encoder,
    layers.Dense(128,activation="relu"),
    layers.Dense(64,activation='relu'),
    layers.Dense(64, activation="relu"),
    layers.Dense(1,activation="sigmoid",name="output_layer")
])



In [99]:
model3.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(),
    metrics=['accuracy']
  )

In [100]:
history3 = model3.fit(
    train_senteces,
    train_labels,
    epochs=10,
    validation_data= (val_senteces,val_labels),
    callbacks=[create_tensorboard_callback("Save.dir",
                                           "USE")]
)

Saving TensorBoard log files to: Save.dir/USE/20210722-012334
Epoch 1/10
215/215 [==============================] - 5s 20ms/step - loss: 0.0712 - accuracy: 0.9750 - val_loss: 0.8853 - val_accuracy: 0.7874
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0610 - accuracy: 0.9793 - val_loss: 0.9411 - val_accuracy: 0.7966
Epoch 3/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0483 - accuracy: 0.9826 - val_loss: 0.9682 - val_accuracy: 0.7874
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0453 - accuracy: 0.9839 - val_loss: 1.0557 - val_accuracy: 0.7848
Epoch 5/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0452 - accuracy: 0.9815 - val_loss: 1.0082 - val_accuracy: 0.7848
Epoch 6/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0369 - accuracy: 0.9847 - val_loss: 1.1923 - val_accuracy: 0.7940
Epoch 7/10
215/215 [==============================] - 2s 10ms/step - loss: 0.036

In [101]:
model3_pred_probs = model3.predict(val_senteces)

model3_pred_probs[:10]

array([[4.0243490e-06],
       [9.3268073e-01],
       [1.0000000e+00],
       [2.9489732e-01],
       [8.5608270e-03],
       [9.9959403e-01],
       [9.9999142e-01],
       [9.9999166e-01],
       [9.9936885e-01],
       [2.5467786e-03]], dtype=float32)

In [102]:
model3_pred = tf.squeeze(tf.round(model3_pred_probs))

In [103]:
model3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [104]:
model3_results = calculate_results(y_true=val_labels,
                                   y_pred=model3_pred)

In [105]:
model3_results

{'accuracy': 79.79002624671917,
 'f1': 0.7946275201865691,
 'precision': 0.8047465799140253,
 'recall': 0.7979002624671916}

In [109]:
model3.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [111]:
model3.fit(
    train_senteces,
    train_labels,
    epochs=10,
    validation_data=(val_senteces,val_labels),
    callbacks=[create_tensorboard_callback("Save.dir",
                                           "USEFineTunening")]
)


Saving TensorBoard log files to: Save.dir/USEFineTunening/20210722-012621
Epoch 1/10
215/215 [==============================] - 4s 16ms/step - loss: 0.0300 - accuracy: 0.9873 - val_loss: 1.2906 - val_accuracy: 0.7940
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0205 - accuracy: 0.9905 - val_loss: 1.3833 - val_accuracy: 0.7953
Epoch 3/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0183 - accuracy: 0.9911 - val_loss: 1.4376 - val_accuracy: 0.7940
Epoch 4/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0168 - accuracy: 0.9921 - val_loss: 1.5109 - val_accuracy: 0.7927
Epoch 5/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0160 - accuracy: 0.9923 - val_loss: 1.5321 - val_accuracy: 0.7953
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0145 - accuracy: 0.9930 - val_loss: 1.6326 - val_accuracy: 0.7913
Epoch 7/10
215/215 [==============================] - 2s 11ms/step -

In [112]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [113]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [114]:
model3.fit(
    train_df,
    validation_data=(val_senteces,val_labels),
    epochs=5,
    callbacks=[create_tensorboard_callback("Save.Dir",
                                           "USEfullData")]
)

Saving TensorBoard log files to: Save.Dir/USEfullData/20210722-012945


ValueError: ignored

In [116]:
len(train_df)

7613

In [119]:
len(train_senteces) + len(val_senteces)

7613

In [118]:
len(train_labels)

6851

In [157]:
train_sentences, test_sentneces, train_labels1,test_labels,  = train_test_split(test_df['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1, #Use 10% of data for validation size
                                                                            random_state=42)

ValueError: ignored

In [156]:
len(train_sentences), len(test_sentneces), len(train_labels1), len(test_labels)

(6851, 762, 6851, 762)

In [158]:
train_sentences_full = train_df["text"].to_numpy()

In [161]:
len(train_sentences_full)

7613

In [162]:
train_labels_full = train_df["target"].to_numpy()

In [164]:
len(train_labels_full)

7613

In [165]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [167]:
test_data_full = test_df["text"].to_numpy()

In [168]:
test_data_full

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [171]:
model3.fit(
    train_sentences_full,
    train_labels_full,
    epochs=5,
    validation_data=(val_senteces,val_labels),
    callbacks=[create_tensorboard_callback("Save.dir",
                                           "Model3_All_data")]
)

Saving TensorBoard log files to: Save.dir/Model3_All_data/20210722-015440
Epoch 1/5
238/238 [==============================] - 3s 13ms/step - loss: 0.1568 - accuracy: 0.9722 - val_loss: 0.7648 - val_accuracy: 0.8556
Epoch 2/5
238/238 [==============================] - 3s 11ms/step - loss: 0.0916 - accuracy: 0.9802 - val_loss: 0.5239 - val_accuracy: 0.8885
Epoch 3/5
238/238 [==============================] - 2s 11ms/step - loss: 0.0724 - accuracy: 0.9827 - val_loss: 0.4176 - val_accuracy: 0.9055
Epoch 4/5
238/238 [==============================] - 3s 11ms/step - loss: 0.0620 - accuracy: 0.9845 - val_loss: 0.3474 - val_accuracy: 0.9226
Epoch 5/5
238/238 [==============================] - 3s 11ms/step - loss: 0.0540 - accuracy: 0.9856 - val_loss: 0.2895 - val_accuracy: 0.9331


In [173]:
model3_pred_probs_final = model3.predict(test_data_full)
model3_pred_probs_final

array([[0.92839515],
       [0.99868816],
       [0.9993562 ],
       ...,
       [0.99999225],
       [0.99881077],
       [0.9942568 ]], dtype=float32)

In [176]:
model3_pred_final = tf.squeeze(tf.round(model3_pred_probs_final))
model3_pred_final

<tf.Tensor: shape=(3263,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [177]:
model3_results_full = calculate_results(val_labels,
                                        model3_pred_final)

ValueError: ignored

In [179]:
len(test_data_full)

3263